# Movie Recommender System

The MovieLens dataset contains 100k movie ratings from 943 users and a selection of 1682 movies. It is downloadable [here](http://files.grouplens.org/datasets/movielens/ml-100k.zip).

### Methods

* Collaborative filtering produces recommendations based on the knowledge of users’ attitude to items (i.e. "wisdom of the crowd")
* Content-based recommender systems focus on the attributes of the items and give you recommendations based on the similarity between them

In [32]:
import numpy as np
import pandas as pd

column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('data/u.data', sep='\t', names=column_names)

In [33]:
df.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


In [34]:
movie_titles = pd.read_csv("data/Movie_Id_Titles")
movie_titles.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [35]:
df = pd.merge(df,movie_titles,on='item_id')
df.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


In [36]:
n_users = df.user_id.nunique()
n_items = df.item_id.nunique()

print('Num. of Users: '+ str(n_users))
print('Num of Movies: '+str(n_items))

Num. of Users: 944
Num of Movies: 1682


## Recommender system 

In [37]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.25)

### Memory-Based Collaborative Filtering

There are two primary methods; we use the latter.

* Item-Item Collaborative Filtering: “Users who liked this item also liked ..."
* User-Item Collaborative Filtering: "Users who are similar to you also liked ...”

We utilize **cosine similarity**, with similarity between users $k$ and $a$ given by:

$s_u^{\cos}(u_k, u_a) = \dfrac{u_k \cdot u_a}{\|u_k\| \|u_a\|} = \dfrac{\sum_m x_{k,m}x_{a,m}}{\sqrt{\sum_m x_{k,m}^2 \sum_m x_{a,m}^2}}$

In both cases, you create a user-item matrix which built from the entire dataset.

Since we have split the data into testing and training we will need to create two ``[943 x 1682]`` matrices (all users by all movies). 

The training matrix contains 75% of the ratings and the testing matrix contains 25% of the ratings.  

In [38]:
from sklearn.metrics.pairwise import pairwise_distances

train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
  train_data_matrix[line[1]-1, line[2]-1] = line[3]  

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
  test_data_matrix[line[1]-1, line[2]-1] = line[3]

user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

User-based CF:

$\hat{x}_{k,m} = \bar{x}_{k} + \dfrac{\sum_{u_a} s_u^{\cos}(u_k,u_a)(x_{a,m}-\overline{x_{u_a}})}{\sum_{u_a}|s_u^{\cos}(u_k,u_a)|}$

The idea of this formula is due to many users' tendencies to give high or low ratings to all movies. Thus, we evaluate the relative rather than absolute difference.

In [39]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [40]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

In [41]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
  prediction = prediction[ground_truth.nonzero()].flatten() 
  ground_truth = ground_truth[ground_truth.nonzero()].flatten()
  return sqrt(mean_squared_error(prediction, ground_truth))

print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 3.127314113648716
Item-based CF RMSE: 3.454582804867164


### Model-based Collaborative Filtering

In [42]:
sparsity=round(1.0-len(df)/float(n_users*n_items),3)
print('Sparsity', str(sparsity*100) + '%')

Sparsity 93.7%


### Singular value decomposition (SVD)

For $X \in \mathbb R^{m \times n}$, we factorize $X = USV^{\intercal}$, where:

* $U \in \mathbb R^{m \times r}$ is orthogonal
* $S \in \mathbb R^{r \times r}$ is diagonal with nonnegative entries
* $V^\intercal \in \mathbb R^{r \times n}$ is orthogonal

The idea is that $U$ represents the feature vectors corresponds to users in the hidden feature space, and $V$ represents the feature vectors corresponding to items in the hidden feature space. The prediction is based on the product $(U \cdot \mathrm{diag}(S)) \cdot V^\intercal$, where $\mathrm{diag} (S) := \begin{pmatrix} | \\ S_{ii} \\ | \end{pmatrix}$ is an $r \times 1$ vector.

This technique was adopted in the winning submission of the [Netflix Prize](http://buzzard.ups.edu/courses/2014spring/420projects/math420-UPS-spring-2014-gower-netflix-SVD.pdf).

In [43]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data_matrix, k = 20)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
print('User-based CF MSE: ' + str(rmse(X_pred, test_data_matrix)))

User-based CF MSE: 2.71648424077992
